### This is a notebook for a simple benchmark of DenseNet. As this is not the main contribution, this notebook has no comments and/or is well organized. It is a proof-of-concept

In [1]:
import tensorflow as tf
import numpy as np

from alphaenas.data_utils import read_data, saveObjWithPickle, createPath

In [2]:
data_path = 'data/cifar10'

In [3]:
images, labels = read_data(data_path)

--------------------------------------------------------------------------------
Reading data
data/cifar10/data_batch_1
data/cifar10/data_batch_2
data/cifar10/data_batch_3
data/cifar10/data_batch_4
data/cifar10/data_batch_5
data/cifar10/test_batch
Prepropcess: [subtract mean], [divide std]
mean: [125.34512 122.94169 113.83898]
std: [63.02383 62.13708 66.74233]


In [4]:
images.keys()

dict_keys(['train', 'valid', 'test'])

In [5]:
'''class ChildModel():
    def __init__(self, images, labels, path):
        tf.reset_default_graph() 
        self.width = images['train'].shape[1]
        self.height = images['train'].shape[2]
        self.channels = images['train'].shape[3]
        self.num_classes = len(np.unique(labels['train']))
        self.path = path
    
    def build_model(self):
        batch_size = 32
        input_data = tf.placeholder(shape=[None, self.width, self.height, self.channels], dtype=tf.float32, name='x_input')
        y_output = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='y_output')
        bl_training = tf.placeholder(tf.bool, name='training')
        
        x = tf.layers.conv2d(input_data, filters = 512, kernel_size = (3, 3), padding = 'same')
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(input_data, filters = 256, kernel_size = (3, 3), padding = 'same')
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(input_data, filters = 128, kernel_size = (3, 3), padding = 'same')
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(input_data, filters = 64, kernel_size = (3, 3), padding = 'same')
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(input_data, filters = 32, kernel_size = (3, 3), padding = 'same')
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, 1024)
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        x = tf.layers.dense(x, 1024)
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        x = tf.layers.dense(x, self.num_classes)
        x_softmax = tf.nn.softmax(x)
        
        y = tf.one_hot(tf.cast(y_output, tf.int32), self.num_classes)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y))
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.MomentumOptimizer(0.01, 0.9, use_nesterov=True).minimize(loss)
        
        self.optimizer = optimizer
        self.loss = loss
        self.input_data = input_data
        self.y_output = y_output
        self.x = x
        self.bl_training = bl_training
        self.x_softmax = x_softmax
        
    def train(self, images, labels):
        optimizer = self.optimizer
        loss = self.loss
        input_data = self.input_data 
        y_output = self.y_output
        x = self.x
        bl_training = self.bl_training
        x_softmax = self.x_softmax
        batch_size = 32
        path = self.path
        
        createPath(path)
        x_train = images['train']
        y_train = labels['train']
        x_test = images['test']
        y_test = labels['test']
        y_train = y_train.reshape((-1, 1))
        y_test = y_test.reshape((-1, 1))
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            idx = np.asarray(range(x_train.shape[0]))
            for e in range(8):
                no_batches = idx.shape[0]//batch_size + 1
                np.random.shuffle(idx)
                for batch in range(no_batches-1):
                    x_train_batch = x_train[idx[(batch*batch_size):(min((1+batch)*batch_size,x_train.shape[0]))]]
                    y_train_batch = y_train[idx[(batch*batch_size):(min((1+batch)*batch_size,x_train.shape[0]))]]
                    o_loss, o_optimizer, o_x = sess.run([loss, optimizer, x], feed_dict={input_data:x_train_batch, y_output:y_train_batch, bl_training: 1})
                
                val_o_loss, val_o_x = sess.run([loss, x_softmax], feed_dict={input_data:x_test, y_output: y_test, bl_training: 0})
                arg_max = np.argmax(val_o_x, axis=-1)
                val_acc = np.mean(arg_max.reshape((-1,1))==y_test)
                print('Epoch: ' + str(e) + ' Val Loss: ' + str(val_o_loss) + ' Val Acc: ' + str(val_acc))
        
'''        

"class ChildModel():\n    def __init__(self, images, labels, path):\n        tf.reset_default_graph() \n        self.width = images['train'].shape[1]\n        self.height = images['train'].shape[2]\n        self.channels = images['train'].shape[3]\n        self.num_classes = len(np.unique(labels['train']))\n        self.path = path\n    \n    def build_model(self):\n        batch_size = 32\n        input_data = tf.placeholder(shape=[None, self.width, self.height, self.channels], dtype=tf.float32, name='x_input')\n        y_output = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='y_output')\n        bl_training = tf.placeholder(tf.bool, name='training')\n        \n        x = tf.layers.conv2d(input_data, filters = 512, kernel_size = (3, 3), padding = 'same')\n        x = tf.layers.batch_normalization(x, training = bl_training)\n        x = tf.nn.relu(x)\n        x = tf.layers.conv2d(input_data, filters = 256, kernel_size = (3, 3), padding = 'same')\n        x = tf.layers.batch_n

In [6]:
#model = ChildModel(images, labels)

In [7]:
#model.build_model()

In [8]:
#model.train(images, labels)

import cv2

IMG_WIDTH = 32
IMG_HEIGHT = 32
IMG_DEPTH = 3
NUM_CLASS = 10

def horizontal_flip(image, axis):
    '''
    Flip an image at 50% possibility
    :param image: a 3 dimensional numpy array representing an image
    :param axis: 0 for vertical flip and 1 for horizontal flip
    :return: 3D image after flip
    '''
    flip_prop = np.random.randint(low=0, high=2)
    if flip_prop == 0:
        image = cv2.flip(image, axis)

    return image


def whitening_image(image_np):
    '''
    Performs per_image_whitening
    :param image_np: a 4D numpy array representing a batch of images
    :return: the image numpy array after whitened
    '''
    for i in range(len(image_np)):
        mean = np.mean(image_np[i, ...])
        # Use adjusted standard deviation here, in case the std == 0.
        std = np.max([np.std(image_np[i, ...]), 1.0/np.sqrt(IMG_HEIGHT * IMG_WIDTH * IMG_DEPTH)])
        image_np[i,...] = (image_np[i, ...] - mean) / std
    return image_np


def random_crop_and_flip(batch_data):
    '''
    Helper to random crop and random flip a batch of images
    :param padding_size: int. how many layers of 0 padding was added to each side
    :param batch_data: a 4D batch array
    :return: randomly cropped and flipped image
    '''
    cropped_batch = np.zeros(len(batch_data) * IMG_HEIGHT * IMG_WIDTH * IMG_DEPTH).reshape(
        len(batch_data), IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)

    for i in range(len(batch_data)):
        x_offset = np.random.randint(low=0, high=2 * 2, size=1)[0]
        y_offset = np.random.randint(low=0, high=2 * 2, size=1)[0]
        cropped_batch[i, x_offset:x_offset+IMG_HEIGHT,
                      y_offset:y_offset+IMG_WIDTH] = batch_data[i, ...][x_offset:x_offset+IMG_HEIGHT,
                      y_offset:y_offset+IMG_WIDTH, :]

        cropped_batch[i, ...] = horizontal_flip(image=cropped_batch[i, ...], axis=1)

    return cropped_batch


class ResNetModel():
    def __init__(self, images, labels, batch_size, max_iteration, lr, lr_iteration_step = [32000, 42000], path):
        tf.reset_default_graph() 
        self.width = images['train'].shape[1]
        self.height = images['train'].shape[2]
        self.channels = images['train'].shape[3]
        self.num_classes = len(np.unique(labels['train']))
        self.batch_size = batch_size
        self.max_iteration = max_iteration
        self.lr_iteration_step = lr_iteration_step
        self.lr = lr
        self.regularizer = tf.contrib.layers.l2_regularizer(scale=0.0002)
        self.path = path
    
    def bn_relu_conv_layer(self,
                           x,
                           bl_training,
                           filter_num, 
                           filter_size, 
                           stride = (1, 1),
                           padding = 'same'
                          ):
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters = filter_num, 
                             kernel_size = filter_size, 
                             strides = stride, 
                             padding = padding,
                             kernel_regularizer=self.regularizer)
        return(x)
    
    def residual_block(self,
                       i,
                       x, 
                       bl_training,
                       filter_num
                      ):
        
        input_channel = x.get_shape().as_list()[-1]
        if i == 0:
            conv = self.bn_relu_conv_layer(x, bl_training, filter_num, (3,3), stride = (2, 2))
        else:
            conv = self.bn_relu_conv_layer(x, bl_training, filter_num, (3,3))
            
        conv = self.bn_relu_conv_layer(conv, bl_training, filter_num, (3,3))
        
        pooled_input = None
        if i == 0:
            pooled_input = tf.nn.avg_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
            padded_input = tf.pad(pooled_input, [[0, 0], [0, 0], [0, 0], [input_channel // 2, input_channel // 2]])
        else:
            padded_input = x

        output = conv + padded_input
        return(output)     
    
    def build_model(self):
        batch_size = self.batch_size
        n = 5
        input_data = tf.placeholder(shape=[None, self.width, self.height, self.channels], dtype=tf.float32, name='x_input')
        y_output = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='y_output')
        bl_training = tf.placeholder(tf.bool, name='training')
        learning_rate = tf.placeholder(tf.float32, shape=[])
        
        x = tf.layers.conv2d(input_data, filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_regularizer=self.regularizer)
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        
        x = tf.layers.conv2d(x, filters = 16, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_regularizer=self.regularizer)
        conv = self.bn_relu_conv_layer(x, bl_training, 16, (3,3))
        x = conv + x
        
        print('First block')
        for i in range(1, n):
            print(i)
            x = self.residual_block(i, x, bl_training, 16)
        
        print('Second block')
        for i in range(n):
            print(i)
            x = self.residual_block(i, x, bl_training, 32)
        
        print('Third block')
        for i in range(n):
            print(i)
            x = self.residual_block(i, x, bl_training, 64)
        
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.nn.relu(x)
        
        x = tf.reduce_mean(x, axis=[1,2])
        x = tf.layers.dense(x, 10)
        x_softmax = tf.nn.softmax(x)
        
        y = tf.one_hot(tf.cast(y_output, tf.int32), self.num_classes)
        l2_loss = tf.losses.get_regularization_loss()
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y))
        loss += l2_loss
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True).minimize(loss)
            #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
            #optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.5).minimize(loss)
        
        self.optimizer = optimizer
        self.loss = loss
        self.input_data = input_data
        self.y_output = y_output
        self.x = x
        self.bl_training = bl_training
        self.x_softmax = x_softmax
        self.learning_rate = learning_rate
        
    def train(self, images, labels):
        optimizer = self.optimizer
        loss = self.loss
        input_data = self.input_data 
        y_output = self.y_output
        x = self.x
        bl_training = self.bl_training
        x_softmax = self.x_softmax
        batch_size = self.batch_size
        max_iteration = self.max_iteration
        lr_iteration_step = self.lr_iteration_step
        lr = self.lr
        learning_rate = self.learning_rate
        path = self.path
        
        createPath(path)
        x_train = images['train']
        y_train = labels['train']
        x_test = images['test']
        y_test = labels['test']
        y_train = y_train.reshape((-1, 1))
        y_test = y_test.reshape((-1, 1))
        
        iteration = 0
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            idx = np.asarray(range(x_train.shape[0]))
            for e in range(100000):
                no_batches = idx.shape[0]//batch_size + 1
                np.random.shuffle(idx)
                for batch in range(no_batches-1):
                    x_train_batch = x_train[idx[(batch*batch_size):(min((1+batch)*batch_size,x_train.shape[0]))]]
                    x_train_batch = random_crop_and_flip(x_train_batch)
                    y_train_batch = y_train[idx[(batch*batch_size):(min((1+batch)*batch_size,x_train.shape[0]))]]
                    o_loss, o_optimizer, o_x = sess.run([loss, optimizer, x], feed_dict={input_data:x_train_batch, 
                                                                                         y_output:y_train_batch, 
                                                                                         bl_training: 1,
                                                                                         learning_rate: lr})
                    iteration += 1
                    if iteration == max_iteration:
                        break
                    if iteration in lr_iteration_step:
                        lr = 0.1 * lr
                        print('New learning rate: ' + str(lr))
                        
                val_o_loss, val_o_x = sess.run([loss, x_softmax], feed_dict={input_data:x_test, y_output: y_test, bl_training: 0})
                arg_max = np.argmax(val_o_x, axis=-1)
                val_acc = np.mean(arg_max.reshape((-1,1))==y_test)
                print('Epoch: ' + str(e) + ' Iteration: ' + str(iteration) + ' Val Loss: ' + str(val_o_loss) + ' Val Acc: ' + str(val_acc))
'''

In [9]:
#model = ResNetModel(images, 
#                    labels,
#                    batch_size = 128, 
#                    max_iteration = 64000, 
#                    lr = 0.1
#                   )

In [10]:
#model.build_model()

In [11]:
#model.train(images, labels)

In [12]:
import cv2
import time
from timeit import default_timer as timer

IMG_WIDTH = 32
IMG_HEIGHT = 32
IMG_DEPTH = 3
NUM_CLASS = 10

def horizontal_flip(image, axis):
    '''
    Flip an image at 50% possibility
    :param image: a 3 dimensional numpy array representing an image
    :param axis: 0 for vertical flip and 1 for horizontal flip
    :return: 3D image after flip
    '''
    flip_prop = np.random.randint(low=0, high=2)
    if flip_prop == 0:
        image = cv2.flip(image, axis)

    return image


def whitening_image(image_np):
    '''
    Performs per_image_whitening
    :param image_np: a 4D numpy array representing a batch of images
    :return: the image numpy array after whitened
    '''
    for i in range(len(image_np)):
        mean = np.mean(image_np[i, ...])
        # Use adjusted standard deviation here, in case the std == 0.
        std = np.max([np.std(image_np[i, ...]), 1.0/np.sqrt(IMG_HEIGHT * IMG_WIDTH * IMG_DEPTH)])
        image_np[i,...] = (image_np[i, ...] - mean) / std
    return image_np


def random_crop_and_flip(batch_data):
    '''
    Helper to random crop and random flip a batch of images
    :param padding_size: int. how many layers of 0 padding was added to each side
    :param batch_data: a 4D batch array
    :return: randomly cropped and flipped image
    '''
    cropped_batch = np.zeros(len(batch_data) * IMG_HEIGHT * IMG_WIDTH * IMG_DEPTH).reshape(
        len(batch_data), IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)

    for i in range(len(batch_data)):
        x_offset = np.random.randint(low=0, high=2 * 2, size=1)[0]
        y_offset = np.random.randint(low=0, high=2 * 2, size=1)[0]
        cropped_batch[i, x_offset:x_offset+IMG_HEIGHT,
                      y_offset:y_offset+IMG_WIDTH] = batch_data[i, ...][x_offset:x_offset+IMG_HEIGHT,
                      y_offset:y_offset+IMG_WIDTH, :]

        cropped_batch[i, ...] = horizontal_flip(image=cropped_batch[i, ...], axis=1)

    return cropped_batch


class DenseNetModel():
    def __init__(self, images, labels, batch_size, max_iteration, lr, path, lr_iteration_step = [15000, 30000]):
        tf.reset_default_graph() 
        self.width = images['train'].shape[1]
        self.height = images['train'].shape[2]
        self.channels = images['train'].shape[3]
        self.num_classes = len(np.unique(labels['train']))
        self.batch_size = batch_size
        self.max_iteration = max_iteration
        self.lr_iteration_step = lr_iteration_step
        self.lr = lr
        self.regularizer = tf.contrib.layers.l2_regularizer(scale=0.0001)   
        self.path = path
    
    def dense_block(self, x, nb_layers, growth_rate, bottleneck, wd, p, bl_training=None):
        if bottleneck: 
            nb_layers //= 2
        for i in range(nb_layers):
            b = self.conv_block(x, growth_rate, bottleneck=bottleneck, p=p, wd=wd, bl_training=bl_training)
            x = tf.concat([x,b], axis=-1)
        return x
    
    def conv_block(self, x, nf, bottleneck=False, p=None, wd=0, bl_training=None):
        x = tf.nn.relu(x)
        x = tf.layers.batch_normalization(x, training = bl_training)
        if bottleneck:
            x = tf.layers.conv2d(x, filters = nf*4, kernel_size = (1, 1), strides = (1, 1), padding = 'same', kernel_regularizer=self.regularizer)
            x = tf.layers.dropout(x, rate=p, training=bl_training)
            x = tf.nn.relu(x)
            x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.layers.conv2d(x, filters = nf, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_regularizer=self.regularizer)
        x = tf.layers.dropout(x, rate=p, training=bl_training)
        return(x)
    
    def transition_block(self, x, compression=1.0, p=None, wd=0, bl_training=None):
        nf = int(x.get_shape().as_list()[-1] * compression)
        x = tf.nn.relu(x)
        x = tf.layers.batch_normalization(x, training = bl_training)
        x = tf.layers.conv2d(x, filters = nf, kernel_size = (1, 1), strides = (1, 1), padding = 'same', kernel_regularizer=self.regularizer)
        x = tf.layers.dropout(x, rate=p, training=bl_training)
        x = tf.layers.average_pooling2d(x, (2, 2), (2, 2))
        return(x)
    
    def build_model(self):
        batch_size = self.batch_size
        n = 5
        depth = 100
        nb_filter = 16
        compression = 0.5
        bottleneck = True
        p = 0.2
        wd = 1e-4
        nb_block=3
        growth_rate=12
        
        input_data = tf.placeholder(shape=[None, self.width, self.height, self.channels], dtype=tf.float32, name='x_input')
        y_output = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='y_output')
        bl_training = tf.placeholder(tf.bool, name='training')
        learning_rate = tf.placeholder(tf.float32, shape=[])
        
        nb_layers_per_block = int((depth - 4) / nb_block)
        nb_layers = [nb_layers_per_block] * nb_block
        
        x = tf.layers.conv2d(input_data, filters = 16, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_regularizer=self.regularizer)
        x = tf.layers.dropout(x, rate=p, training=bl_training)
        
        for i, block in enumerate(nb_layers):
            x = self.dense_block(x, block, growth_rate, bottleneck=bottleneck, p=p, wd=wd, bl_training=bl_training)
            if i != len(nb_layers)-1:
                x = self.transition_block(x, compression=compression, p=p, wd=wd, bl_training=bl_training)
        
        x = tf.nn.relu(x)
        x = tf.layers.batch_normalization(x, training = bl_training)
                
        x = tf.reduce_mean(x, axis=[1,2])
        x = tf.layers.dense(x, 10, kernel_regularizer=self.regularizer)
        x_softmax = tf.nn.softmax(x)
        
        y = tf.one_hot(tf.cast(y_output, tf.int32), self.num_classes)
        l2_loss = tf.losses.get_regularization_loss()
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y))
        loss += l2_loss
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True).minimize(loss)
            #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
            #optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.5).minimize(loss)
        
        self.optimizer = optimizer
        self.loss = loss
        self.input_data = input_data
        self.y_output = y_output
        self.x = x
        self.bl_training = bl_training
        self.x_softmax = x_softmax
        self.learning_rate = learning_rate
        
    def predict_batch(self, sess, tensors, input_data, y_output, bl_training, tmp_images, tmp_labels, batch_size):
        idx_tmp = np.asarray(range(tmp_images.shape[0]))
        no_batches_tmp = idx_tmp.shape[0]//batch_size + 1
        y_tmp_final = []
        x_tmp_pred_final = []
        tmp_o_loss_final = []
        for batch in range(no_batches_tmp-1):
            x_tmp_batch = tmp_images[idx_tmp[(batch*batch_size):(min((1+batch)*batch_size,tmp_images.shape[0]))]]
            y_tmp_batch = tmp_labels[idx_tmp[(batch*batch_size):(min((1+batch)*batch_size,tmp_images.shape[0]))]]
            tmp_o_loss, tmp_o_x = sess.run(tensors, feed_dict={input_data:x_tmp_batch, y_output: y_tmp_batch, bl_training: 0})
            arg_max = np.argmax(tmp_o_x, axis=-1)
            x_tmp_pred_final.append(arg_max)
            y_tmp_final.append(y_tmp_batch)
            tmp_o_loss_final.append(tmp_o_loss)
            
        return(y_tmp_final, x_tmp_pred_final, tmp_o_loss_final)
                
        y_test_final = np.asarray(y_test_final)
        x_pred_final = np.asarray(x_pred_final)
        val_acc = np.mean(x_pred_final.reshape((-1,1))==y_test_final.reshape((-1,1)))
        val_o_loss = np.mean(val_o_loss)
    
    def train(self, images, labels):
        optimizer = self.optimizer
        loss = self.loss
        input_data = self.input_data 
        y_output = self.y_output
        x = self.x
        bl_training = self.bl_training
        x_softmax = self.x_softmax
        batch_size = self.batch_size
        max_iteration = self.max_iteration
        lr_iteration_step = self.lr_iteration_step
        lr = self.lr
        learning_rate = self.learning_rate
        path = self.path
        
        createPath(path)
        createPath(path + '/model')
        x_train = images['train']
        y_train = labels['train']
        x_val = images['valid']
        y_val = labels['valid']
        x_test = images['test']
        y_test = labels['test']
        y_train = y_train.reshape((-1, 1))
        y_val = y_val.reshape((-1, 1))
        y_test = y_test.reshape((-1, 1))
        
        iteration = 0
        iteration_total = []
        val_acc_total = []
        test_acc_total = []
        total_time_total = []
        result = {}
        
        best_loss = 999
        best_epoch = 0
        bl_break = False
        
        with tf.Session() as sess:
            saver = tf.train.Saver()
            sess.run(tf.global_variables_initializer())
            idx = np.asarray(range(x_train.shape[0]))
            for e in range(100000):
                if bl_break:
                    break
                start = timer()
                no_batches = idx.shape[0]//batch_size + 1
                np.random.shuffle(idx)
                for batch in range(no_batches-1):
                    x_train_batch = x_train[idx[(batch*batch_size):(min((1+batch)*batch_size,x_train.shape[0]))]]
                    x_train_batch = random_crop_and_flip(x_train_batch)
                    y_train_batch = y_train[idx[(batch*batch_size):(min((1+batch)*batch_size,x_train.shape[0]))]]
                    o_loss, o_optimizer, o_x = sess.run([loss, optimizer, x], feed_dict={input_data:x_train_batch, 
                                                                                         y_output:y_train_batch, 
                                                                                         bl_training: 1,
                                                                                         learning_rate: lr})
                    iteration += 1
                    if iteration == max_iteration:
                        bl_break = True
                    if iteration in lr_iteration_step:
                        lr = 0.1 * lr
                        print('New learning rate: ' + str(lr))
                                        
                y_val_final, x_val_pred_final, val_o_loss_final = self.predict_batch(sess, 
                                                                                     [loss, x_softmax],
                                                                                     input_data, y_output, bl_training,
                                                                                     x_val, 
                                                                                     y_val, 
                                                                                     batch_size) 
                
                y_val_final = np.asarray(y_val_final)
                x_val_pred_final = np.asarray(x_val_pred_final)
                val_acc = np.mean(x_val_pred_final.reshape((-1,1))==y_val_final.reshape((-1,1)))
                val_o_loss_mean = np.mean(val_o_loss_final)
                
                y_test_final, x_test_pred_final, test_o_loss_final = self.predict_batch(sess, 
                                                                                        [loss, x_softmax],
                                                                                        input_data, y_output, bl_training,
                                                                                        x_test, 
                                                                                        y_test, 
                                                                                        batch_size) 
                
                y_test_final = np.asarray(y_test_final)
                x_test_pred_final = np.asarray(x_test_pred_final)
                test_acc = np.mean(x_test_pred_final.reshape((-1,1))==y_test_final.reshape((-1,1)))
                test_o_loss_mean = np.mean(test_o_loss_final)
                
                if best_loss > val_o_loss_mean:
                    print('Safe best model')
                    best_loss = val_o_loss_mean
                    best_epoch = e
                    saver.save(sess, path + '/model/{}'.format('best_model' + str(best_epoch)))
                
                iteration_total.append(iteration)
                val_acc_total.append(val_acc)
                test_acc_total.append(test_acc)
                
                end = timer()
                total_time = end-start
                total_time_total.append(total_time)
                result['iteration_total'] = iteration_total
                result['val_acc'] = val_acc_total
                result['test_acc'] = test_acc_total
                result['time'] = total_time_total
                
                saveObjWithPickle(result, path + '/result.pickle')
                
                print('Time: ' + str(total_time) + ' Epoch: ' + str(e) + ' Iteration: ' + str(iteration) + ' Val Loss: ' + str(val_o_loss_mean) + ' Val Acc: ' + str(val_acc))


In [13]:
createPath('exp/densenet_variance')

for no_model in range(1, 10):
    createPath('exp/densenet_variance/' + str(no_model))
    model = DenseNetModel(images, 
                          labels,
                          batch_size = 128, 
                          max_iteration = 15000, 
                          lr = 0.1,
                          lr_iteration_step = [7500, 10000],
                          path = 'exp/densenet_variance/' + str(no_model)
                         )
    model.build_model()
    model.train(images, labels)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Safe best model
Time: 431.8055562200025 Epoch: 0 Iteration: 351 Val Loss: 1.580794 Val Acc: 0.5667067307692307
Safe best model
Time: 358.7637650610013 Epoch: 1 Iteration: 702 Val Loss: 1.2038969 Val Acc: 0.6822916666666666
Safe best model
Time: 358.8891367239994 Epoch: 2 Iteration: 1053 Val Loss: 1.1208892 Val Acc: 0.7069310897435898
Time: 355.477563092998 Epoch: 3 Iteration: 1404 Val Loss: 1.1801211 Val Acc: 0.6935096153846154
Safe best model
Time: 359.3599541420008 Epoch: 4 Iteration: 1755 Val Loss: 0.93183994 Val Acc: 0.7580128205128205
Safe best model
Time: 358.5475252969991 Epoch: 5 Iteration: 2106 Val Loss: 0.82710904 Val Acc: 0.797676282051282
Time: 355.6350153050007 Epoch: 6 Iteration: 2457 Val Loss: 0.84352964 Val Acc: 0.7938701923076923
Time: 355.6067929439996 Epoch: 7 Iteration: 2808 Va

Time: 354.538715607996 Epoch: 28 Iteration: 10179 Val Loss: 0.509103 Val Acc: 0.8968349358974359
Safe best model
Time: 357.9416380729963 Epoch: 29 Iteration: 10530 Val Loss: 0.5057621 Val Acc: 0.8984375
Safe best model
Time: 358.006988096 Epoch: 30 Iteration: 10881 Val Loss: 0.5050799 Val Acc: 0.8970352564102564
Safe best model
Time: 358.0606388249944 Epoch: 31 Iteration: 11232 Val Loss: 0.50419706 Val Acc: 0.8984375
Safe best model
Time: 358.03512892599974 Epoch: 32 Iteration: 11583 Val Loss: 0.5040655 Val Acc: 0.8998397435897436
Safe best model
Time: 357.93949780899857 Epoch: 33 Iteration: 11934 Val Loss: 0.50227076 Val Acc: 0.9002403846153846
Safe best model
Time: 358.07272951699997 Epoch: 34 Iteration: 12285 Val Loss: 0.50181866 Val Acc: 0.9006410256410257
Time: 354.5665509620012 Epoch: 35 Iteration: 12636 Val Loss: 0.5020661 Val Acc: 0.8998397435897436
Safe best model
Time: 357.9736905000027 Epoch: 36 Iteration: 12987 Val Loss: 0.5006972 Val Acc: 0.8982371794871795
Time: 354.67223

Time: 356.8664163570065 Epoch: 17 Iteration: 6318 Val Loss: 0.6466306 Val Acc: 0.8485576923076923
Safe best model
Time: 360.25066624599276 Epoch: 18 Iteration: 6669 Val Loss: 0.6276665 Val Acc: 0.8623798076923077
Time: 357.2041127820121 Epoch: 19 Iteration: 7020 Val Loss: 0.6341772 Val Acc: 0.8609775641025641
Time: 357.2190784500126 Epoch: 20 Iteration: 7371 Val Loss: 0.6366002 Val Acc: 0.8501602564102564
New learning rate: 0.010000000000000002
Safe best model
Time: 360.6728262559918 Epoch: 21 Iteration: 7722 Val Loss: 0.538424 Val Acc: 0.8886217948717948
Safe best model
Time: 360.52387011999963 Epoch: 22 Iteration: 8073 Val Loss: 0.51694477 Val Acc: 0.8952323717948718
Safe best model
Time: 360.81180646699795 Epoch: 23 Iteration: 8424 Val Loss: 0.51477605 Val Acc: 0.8974358974358975
Time: 357.3222439300007 Epoch: 24 Iteration: 8775 Val Loss: 0.5173221 Val Acc: 0.8964342948717948
Safe best model
Time: 360.4113720000023 Epoch: 25 Iteration: 9126 Val Loss: 0.5079282 Val Acc: 0.8984375
Tim